# Principal plane

## Definitions

Definitions from Chapter 3 of "Lens Design Fundamentals"

- Equivalent refracting locus
- the principal planes and the principal points
- the focal points
- definition of front / rear focal length

Computing a principal points and focal points should be possible when:
* there is no defined light source nor fixed image plane
* focal length is not known (because the system doesn't have any paraxial information)

Computing the intersection of input and output rays to a sequential system is easy. The difficult part is that the principal point is defined as the limit of that intersection as the incident ray height goes to zero. The intersection itself is undefined at zero.

To do this, we fit a parabola $X = AY^2 + C$ to intersection points computed for rays with a low height value. The chosen height value should be as close as possible to the principal axis, but not so small that numerical precision issues start to arise.

Then, the principal point is the vertex of the best fit parabola: $X = C$.

Same for the focal points, but intersection is between output rays and optical axis.

## Inverting a sequential system

Problem:

how to compute the front principal point? Need a way to "invert" an optical sequence.
* Invert the optical sequence
* Keep or invert the kinematic sequence, both options possible

Insight: the kinematic sequence doesn't depend on the optical one
but the optical sequence depends on the kinematic one, because surface positions / tf are on the kinematic chain

Idea 1: evaluate the kinematic chain first, this creates a realization of the chain, so positions are known
Can now evaluate the optical chain, forward or backwards

kinematic data = just tf chain
kinematic elements = operate on kinematic data

optical data = just ray info
optical elements = operate on optical data, also gets tf info somehow

collision surface is both

1. evaluate kinematic model with forward_tree or similar
2. evaluate optics model, giving realized tf

> how to associate realized tf to optical elements?

using forward tree, I can associate the outputs of a sequential of kinematic element to module in the sequence
but need to associate to modules of another sequence, the optical sequence

indexing probably the best: straightforward, reversible

optical data contains static info on the realized tf chain: a list of length N
what if we have a non linear tf tree, like a fork?

> general case

first pass: evaluate kinematic model
get a realized tf tree, bound to variables
second pass: evaluate the optical model, passing ray data and needed tf data

> sequential special case

first pass: evaluate kinematic model, results in a list of list of tf, length N
the realized tf tree is stored in a list (list of list to handle absolute transform resets)
second pass: evaluate the optical model, passing ray data and tf list elements


Kinematic Elements:
def forward(chain: KinematicChain) -> KinematicChain

Optical Element:
def forward(rays: RayBundle, tf: TransformBase) -> RayBundle

how to do collision surface?
one module should be able to act as both kinematic and optical
forward_kinematic()
forward_optical()

collision surface actually output two tf:
* the one that applies to the held surface
* the one that applis to the next element in the kinematic chain

or fully split collision surface:

SurfaceAnchor: KinematicElement
    def forward(chain: KinematicChain) -> tuple[KinematicChain, KinematicChain]

RefractiveOpticalSurface: OpticalElement
    def forward(rays: RaysBundle, tf: TransformBase) -> RayBundle
    
class RefractiveSurface:
    def init:
        self.anchor_surface
        self.collision_surface
        self.material
    
    def sequential(inputs):
        # kinematic step:
            # compute tf using anchor surface
        # optical step:
            # compute collisions using collision surface at tf
            # compute refraction
        return outputs

        # split sequential() into kinematic step and optical step?
        # outputs of the kinematic step are given as input to the optical step

        # this enables reverse optical mode:
        #    call sequential kinematic steps in normal order
        #    store outputs indexed by the module
        #    call optical steps in reverse order, getting the correct tf inputs for each optical step


in general: elements forward functions do what makes sense
sequential element figures it out to make sequential mode convinient / or elements have a sequential() version of their forward function

> how to use mixed style elements into a sequential?

sequential can have logic:
    if (kinematic element)
        give it tf
    if (optical)
        give it rays
    if (mixed)
        give it both

Sequential elements can contribute to either or both chains

complete model is always a kinematic model and an optical model

inverting = invert the optical model, keep the kinematic model


"Main loop":
    Forward kinematic chain
    -> Obtain a tf for each element of the optical chain
    Forward optical chain

A mixed element is an alias constructor for putting an element on both chains

sequential syntax returns a SequentialModel
that's made of a kinematic and optical model

```
tlm.RefractiveSurface(...)  # returns a kinematic and an optical model


class Sequential:
    kinematic model: Sequential of KinematicElement
    optical model: Sequential of OpticalElement
    # how to give the right tf to each optical element?
    # both have same length, use index
    # or optical element is stored next to an index of corresponding kinematic element
```


In [ ]:
import torch
import torchlensmaker as tlm

doublet = tlm.Sequential(
    tlm.RefractiveSurface(tlm.Sphere(4.0, C=0.135327), material=tlm.NonDispersiveMaterial(1.517)),
    tlm.Gap(1.05),
    tlm.RefractiveSurface(tlm.Sphere(4.0, C=-0.19311), material=tlm.NonDispersiveMaterial(1.649)),
    tlm.Gap(0.4),
    tlm.RefractiveSurface(tlm.Sphere(4.0, C=-0.06164), material="air"),
)


def locus_samples(lens, beam_diameter):
    source = tlm.PointSourceAtInfinity(beam_diameter, sampler_pupil=tlm.LinspaceSampler1D(30), sampler_wavelength=tlm.ZeroSampler1D())
    inputs = source(tlm.default_input(dim=2))
    outputs = doublet(inputs)

    # could also make a sequential system
    # eval it and extract input and output rays
    # or a mix of both because light source is deterministic

    return inputs, outputs, tlm.Sequential(tlm.Gap(-2), source, tlm.Gap(2), doublet)

inputs, outputs, optics = locus_samples(doublet, 2.0)

In [ ]:
def locus(Pa, Va, Pb, Vb):
    """
    """

    # V matrix such that Vt = P
    V = torch.stack((Va, Vb), dim=-1)
    P = Pb - Pa

    return torch.linalg.solve(V, P)

# intersection solution
t = locus(inputs.P, inputs.V, outputs.P, outputs.V)
print(t)

# collision points
Q, W = outputs.P, outputs.V
CP = Q + -t[:, 1].unsqueeze(-1).expand_as(W)*W

scene = tlm.render_sequence(optics, 2, end=3)
scene["data"].append(tlm.render_points(CP, radius=0.01))
tlm.display_scene(scene)

In [ ]:
import matplotlib.pyplot as plt

Py = inputs.P[:, 1]
Qy = outputs.P[:, 1]
Wy = outputs.V[:, 1]

assert torch.isfinite(Py).all()
assert torch.isfinite(Qy).all()
print(Wy)
assert torch.isfinite(Wy).all()

# TODO filter Wy = 0 rays

s = (Py - Qy) / Wy

assert torch.isfinite(s).all()

# collision points
Q = outputs.P
W = outputs.V

CP = Q + s.unsqueeze(-1).expand_as(W)*W

print(CP)

plt.scatter(CP[:, 0], CP[:, 1], s=1.0)
#plt.gca().set_aspect("equal")
plt.gca().margins(1.5, 1.5)

N = Q.shape[0]

x = CP[:, 0]
y = CP[:, 1]

sx = x.sum()
sy4 = (y**4).sum()
sy2 = (y**2).sum()
sy2x = (y**2 * x).sum()

num = sx * (sy4 / sy2) - sy2x
denom = N * (sy4 / sy2) - sy2

print(num, denom)

C = num / denom

print(C)

plt.axvline(C)

